In [1]:
import requests
import json
import pandas as pd

In [2]:
# weibo_id
comment_parameter = []
# weibo_url
comment_url = []

#获取每条微博的id值
url = 'https://m.weibo.cn/api/container/getIndex?uid=1613357522&luicode=10000011&lfid=100103type%3D1%26q%3D%E6%9D%8E_%E8%82%96%E9%81%A5&featurecode=20000320&containerid=1076031613357522'
c_r = requests.get(url)
for i in range(2,11):
    c_parameter = (json.loads(c_r.text)["data"]["cards"][i]["mblog"]["id"])
    print(c_parameter)
    comment_parameter.append(c_parameter)

#获取每条微博评论url
c_url_base = 'https://m.weibo.cn/api/comments/show?id='
for parameter in comment_parameter:
    for page in range(1,101):#提前知道每条微博只可抓取前100页评论
        c_url = c_url_base + str(parameter) + "&page=" + str(page)
        comment_url.append(c_url)


4259588822478446
4259268163488186
4258953711990333
4258837148144524
4257995770574946
4257843693343695
4257468479462315
4257305480202836
4257069538006156


上面获取到每条微博评论的url以后，我们就可以直接请求对应的url，然后把user_id和text解析出来即可

In [ ]:
user_id = []
comment = []

for url in comment_url:
    u_c_r = requests.get(url)
    try:
        for m in range(0,9):#提前知道每个url会包含9条用户信息
            one_id = json.loads(u_c_r.text)["data"]["data"][m]["user"]["id"]
            user_id.append(one_id)
            one_comment = json.loads(u_c_r.text)["data"]["data"][m]["text"]
            comment.append(one_comment)
    except:
        pass

获取containerid

In [ ]:
containerid = []
user_base_url = "https://m.weibo.cn/api/container/getIndex?type=uid&value="
for id in set(user_id):#需要对user_id去重
    containerid_url = user_base_url + str(id)
    try:
        con_r = requests.get(containerid_url)
        one_containerid = json.loads(con_r.text)["data"]['tabsInfo']['tabs'][0]["containerid"]
        containerid.append(one_containerid)
    except:
        containerid.append(0)

获取用户基本信息

In [ ]:
#这里需要设置headers以及cookie模拟登陆
feature = []#存放用户基本信息
id_lose = []#存放请求失败id
user_agent = "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36"
headers = {"User-Agent":user_agent}
m = 1
for num in zip(user_id,containerid):
    url = "https://m.weibo.cn/api/container/getIndex?uid="+str(num[0])+"&luicode=10000011&lfid=100103type%3D1%26q%3D&featurecode=20000320&type=uid&value="+str(num[0])+"&containerid="+str(num[1])
    try:
        r = requests.get(url,headers = headers,cookies = cookie)
        feature.append(json.loads(r.text)["data"]["cards"][1]["card_group"][1]["item_content"].split("  "))
        print("成功第{}条".format(m))
        m = m + 1
        time.sleep(1)#设置睡眠一秒钟，防止被封
    except:
        id_lose.append(num[0])

#将featrue建立成DataFrame结构便于后续分析
user_info = pd.DataFrame(feature,columns = ["性别","年龄","星座","国家城市"])